In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transform
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm
import utils

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
def get_high_low_loaders(data_set):
  train_idx_h = [h for h in range(len(data_set.targets)) if data_set.targets[h] == 0]
  train_idx_l = [l for l in range(len(data_set.targets)) if data_set.targets[l] == 1]

  # high resolution
  train_set_h = torch.utils.data.Subset(data_set, train_idx_h)
  train_loader_h = torch.utils.data.DataLoader(dataset=train_set_h, batch_size=batch_size, shuffle=False)

  # low resolution
  train_set_l = torch.utils.data.Subset(data_set, train_idx_l)
  train_loader_l = torch.utils.data.DataLoader(dataset=train_set_l, batch_size=batch_size, shuffle=False)

  return train_loader_h, train_loader_l


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

batch_size = 32

img_transform = transform.Compose([transform.ToTensor()]) 

# Training Set
train_set =  torchvision.datasets.ImageFolder('/content/drive/MyDrive/UTEC/ciclo2/Aprendizaje Automático/Semana13_PROY4_Autoencoders/dataset/train/', img_transform)
# train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, pin_memory=True)
print(train_set)

# Testing Set
test_set =  torchvision.datasets.ImageFolder('/content/drive/MyDrive/UTEC/ciclo2/Aprendizaje Automático/Semana13_PROY4_Autoencoders/dataset/val/', img_transform)
# test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True, pin_memory=True)
print(test_set)

cuda:0
Dataset ImageFolder
    Number of datapoints: 1370
    Root location: /content/drive/MyDrive/UTEC/ciclo2/Aprendizaje Automático/Semana13_PROY4_Autoencoders/dataset/train/
    StandardTransform
Transform: Compose(
               ToTensor()
           )
Dataset ImageFolder
    Number of datapoints: 340
    Root location: /content/drive/MyDrive/UTEC/ciclo2/Aprendizaje Automático/Semana13_PROY4_Autoencoders/dataset/val/
    StandardTransform
Transform: Compose(
               ToTensor()
           )


In [5]:
_features = 40

In [6]:
utils.size_output_layer(256,3,2,1)

128

In [7]:
utils.size_output_layer(128,3,2,1)

64

In [8]:
class Encoder(nn.Module):
  def __init__(self):
    super(Encoder, self).__init__()
    self.conv1 = nn.Sequential(
      nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1), #salida 
      #nn.BatchNorm2d(64),#out_channels
      nn.ReLU()
    )
    self.conv2 = nn.Sequential(
      nn.Conv2d(in_channels=64, out_channels=64*2, kernel_size=4, stride=2, padding=1), #salida 128
      #nn.BatchNorm2d(64*2),
      nn.ReLU()
    )

    self.conv3 = nn.Sequential(
      nn.Conv2d(in_channels=64*2, out_channels=64*2*2, kernel_size=4, stride=2, padding=1), # salida 32
      #nn.BatchNorm2d(64*2*2),
      nn.ReLU()
    )

    self.fc = nn.Linear(in_features=64*2*2*64*64, out_features=_features)

  def forward(self, image):
    self.out1 = F.relu(self.conv1(image))
    self.out2 = F.relu(self.conv2(self.out1))
    self.out3 = F.relu(self.conv3(self.out2))
    self.out4 = self.out3.view(self.out3.size(0), -1)
    z = self.fc(self.out4)
    return z

class Decoder(nn.Module):
  def __init__(self):
    super(Decoder, self).__init__()
    self.fc    = nn.Linear(in_features=_features,out_features=64*2*2*64*64)
    self.convTran1 = nn.Sequential(
        nn.ConvTranspose2d(in_channels=2*2*2*64,out_channels=64*2, kernel_size=4, stride=2, padding=1),
        #nn.BatchNorm2d(64*2),
        nn.ReLU()
    )
    self.convTran2 = nn.Sequential(
        nn.ConvTranspose2d(in_channels=2*64*2,out_channels=64, kernel_size=4, stride=2, padding=1),
        #nn.BatchNorm2d(64),
        nn.ReLU()
    )
    self.convTran3 = nn.Sequential(
        nn.ConvTranspose2d(in_channels=2*64,out_channels=3, kernel_size=3, stride=1, padding=1),
        #nn.BatchNorm2d(3),
        nn.ReLU()
    )
   
  def forward(self, latent, encoder):
    out = self.fc(latent)
    out = out.view(out.size(0), 64*2*2, 64, 64)
    # out = F.relu(self.convTran1(out))
    out = F.relu(self.convTran1(torch.cat([out, encoder.out3], dim=1)))

    out = F.relu(self.convTran2(torch.cat([out, encoder.out2], dim=1)))

    # out = torch.tanh(self.convTran2(out))
    out = torch.tanh(self.convTran3(torch.cat([out, encoder.out1], dim=1)))

    return out


In [9]:
class Autoencoder(nn.Module):
   def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

   def forward(self, x):
        latent = self.encoder(x)
        x_recon = self.decoder(latent, self.encoder)
        return  x_recon

In [10]:
def train(model, images_set, Epochs, loss_fn):

    images_high_loader, images_low_loader = get_high_low_loaders(images_set)

    train_loss_avg = []
    for epoch in range(Epochs):
      train_loss_avg.append(0)
      num_batches = 0

      images_high_it = iter(images_high_loader)
    
      for image_batch, _ in tqdm(images_low_loader):
          image_batch = image_batch.to(device)
          
          image_batch_recon = model(image_batch)

          image_batch_high, _ = next(images_high_it)
          image_batch_high = image_batch_high.to(device)
          
          loss = loss_fn(image_batch_recon, image_batch_high)
          
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          
          train_loss_avg[-1] += loss.item()
          num_batches += 1
          
      train_loss_avg[-1] /= num_batches
      print('Epoch [%d / %d] average reconstruction error: %f' % (epoch+1, Epochs, train_loss_avg[-1]))
    return train_loss_avg

In [ ]:
learning_rate = 0.001
autoencoder = Autoencoder()
autoencoder.to(device)
loss = nn.MSELoss()
optimizer = torch.optim.Adam(params=autoencoder.parameters(), lr=learning_rate, weight_decay=1e-5)

autoencoder.train()

loss_result = train(autoencoder, train_set, 10, loss)

100%|██████████| 22/22 [00:52<00:00,  2.37s/it]


Epoch [1 / 10] average reconstruction error: 0.125514


100%|██████████| 22/22 [00:50<00:00,  2.30s/it]


Epoch [2 / 10] average reconstruction error: 0.094213


100%|██████████| 22/22 [00:50<00:00,  2.30s/it]


...

100%|██████████| 22/22 [00:50<00:00,  2.30s/it]


Epoch [7 / 10] average reconstruction error: 0.091286


100%|██████████| 22/22 [00:50<00:00,  2.29s/it]


Epoch [8 / 10] average reconstruction error: 0.091166


 23%|██▎       | 5/22 [00:11<00:39,  2.32s/it]

In [ ]:
fig = plt.figure()
plt.plot(loss_result)
plt.xlabel('Epochs')
plt.ylabel('Reconstruction error')
plt.show()

In [ ]:
test_high_loader, test_low_loader = get_high_low_loaders(test_set)

autoencoder.eval()

with torch.no_grad():
  image, label = iter(test_low_loader).next()

  image = image.to(device)
  label = label.to(device)

  z = autoencoder.encoder(image)
  z = z.to(device)

  decodificado = autoencoder.decoder(z, autoencoder.encoder)
  decodificado = decodificado.to('cpu')

In [ ]:
idx_img = 1;
images_l, _ = iter(test_low_loader).next()
images_h, _ = iter(test_high_loader).next()

fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=True, figsize=(20, 20))
ax0.set_title('Test Image (low resolution)')
ax0.imshow(transform.ToPILImage()(images_l[idx_img]))

ax1.set_title('Predicted Image')
ax1.imshow(transform.ToPILImage()(decodificado[idx_img]))

ax2.set_title('Original Image (high resolution)')
ax2.imshow(transform.ToPILImage()(images_h[idx_img]))